In [1]:
import comp_mod_ase as cma
import potfit_reader as pr
from pprint import pprint
import opt_algos as oa

# импорты для визуализации
from ase.visualize import view
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms
from ase.calculators.abc import ABC


In [4]:
data = pr.reader("train.force")

#0:80
#80:127
#127:167
#167:186
# In order to have all kind of structures in datasets

train_data = data[:70] + data[80:120] + data[127:160] + data[167:180]
test_data = data[70:80] + data[120:127] + data[160:167] + data[180:]

print(len(train_data), len(test_data))

156 29


In [5]:
print(type(train_data), type(train_data[0]))
print()
pprint(pr.Structure.__annotations__)

<class 'list'> <class 'potfit_reader.Structure'>

{'atom_types': list[str],
 'cell': <class 'potfit_reader.Cell'>,
 'energy': <class 'float'>,
 'forces': <class 'numpy.ndarray'>,
 'positions': <class 'numpy.ndarray'>}


## EAM

The expression for the interaction potential energy in case of the EAM interatomic potential (for the single component) can be written as follows:
$$
    E_{pe} = \frac{1}{2}\sum_{i \neq j} \Phi(r_{ij}) + \sum_{i} F(n_i),
$$
where electron density for an $i$-th atom has the following form:

$$
    n_i = \sum_{j\neq i} \rho(r_{ij}).
$$

We will use the following functions:

Morse for $\Phi(r)$:
$$
\Phi(r) = D_e ([1-\exp(-a(r-re))]^2 - 1)
$$

csw2 for $\rho(r)$:
$$
\rho(r) = \frac{(1 + a_1 \cos(\alpha r + \phi))}{r^\beta}
$$

bjs for $F(n)$:
$$
F(n) = F_0[1−\gamma\ln n]n^\gamma + F_1 n
$$

In [32]:
pprint(cma.init_eam.__annotations__)

{'F_rho': <built-in function array>,
 'Phi_r': <built-in function array>,
 'Rho_r': <built-in function array>,
 'elements': list[str],
 'masses': list[float],
 'rcut': <class 'float'>,
 'return': <class 'ase.calculators.eam.EAM'>}


In [33]:
# borders

# rcut [4.0, 6.5]

# morse
# de, a, re [[0.02, 1.0],[0.5, 2.0],[3.0, 4.5]]

# csw2 
# a1, alpha, phi, beta [[-1, 1], [0.1, 3.0], [0, np.pi], [0.5, 3.0]]

# bjs
# F0, gamma, F1 [[-4, -1], [0.5, 3.0], [-0.8, 0.5]]

In [18]:
# r = np.linspace(0.0, 6.0, 100)
# rho = np.linspace(0.0, 10.0, 100)
pot = cma.EAM(potential="Lead.eam.alloy")

In [65]:
%time for structure in test_data: cma.comp_fe_ase(pot, structure)

CPU times: user 22.2 s, sys: 1.39 s, total: 23.6 s
Wall time: 5.91 s


## Задача

1. Обучить EAM потенциал по данным `train_data`. Использовать алгоритм из `oa` (наверное, лучше генетический). Надо реализовать loss фунцкцию. Для расчтетов сил и энергий использовать `cma.comp_fe_ase`

Примечание: можно считать, что ошибка $0.1~eV/\AA \Leftrightarrow 10~meV/atom$. Также заметьте, что значения энергий даны для всей системы. Размерность сил $eV/\AA$, а энергий $eV$.

In [9]:
import numpy as np

In [49]:
def loss_func(comp_structures: list[pr.Structure], valid_structures: list[pr.Structure],
        f_weight = 1.0, en_weight = 10.0
    ) -> float:
    # MSE(forces)
    # MSE(energies) do not forget to compute per atom values
    if len(comp_structures) != len(valid_structures):
        raise ValueError('Different lens of arrays')

    comp_forces = np.concatenate([struct.forces for struct in comp_structures])
    valid_forces = np.concatenate([struct.forces for struct in valid_structures])
    forces_mse = np.sum((comp_forces - valid_forces)**2) / comp_forces.size

    comp_energies = np.array([struct.energy / len(struct.positions) for struct in comp_structures])
    valid_energies = np.array([struct.energy / len(struct.positions) for struct in valid_structures])
    energies_mse = np.sum((comp_energies - valid_energies)**2) / comp_energies.size

    loss = np.sqrt(forces_mse)*f_weight + np.sqrt(energies_mse)*en_weight
    return loss

In [15]:
np.sum([[[2, 1], [1, 5]], [[2, 1], [1, 5]]])
a = np.array([[[2, 1], [1, 5]], [[2, 1], [1, 5]]])
a.size


18

In [6]:
pr.Structure.__annotations__

{'cell': potfit_reader.Cell,
 'positions': numpy.ndarray,
 'forces': numpy.ndarray,
 'energy': float,
 'atom_types': list[str]}

In [19]:
comp_structures = [cma.comp_fe_ase(pot, structure) for structure in test_data]


In [46]:
a = [struct.forces for struct in test_data]
type(a[0])
# a = np.array(a)

numpy.float64

In [50]:
loss_func(comp_structures, test_data)

881.010391257251

2. Вывести ошибки на  `train_data` и `test_data`.

3. Построить полученные функции EAM потенциала

4*. Изучить зависимость ошибки от $rcut$.